In [1]:
# !pip install fastapi nest-asyncio pyngrok uvicorn aiofiles python-multipart

In [2]:
# !mkdir templates

In [3]:
# p install aiofiles python-multipart

In [4]:
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import FileResponse
from typing import List
import os
import uvicorn

In [5]:
from fastapi import FastAPI, Request, Form
from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates
import nest_asyncio
from pyngrok import ngrok

import uvicorn
# from fastapi.staticfiles import StaticFiles

In [6]:
%%writefile templates/inputnews2.html
<!DOCTYPE html>
<html>
<head>
<meta charset="UTF-8">
<title>뉴스 줄넘귀</title>
<style>
body {background-color: lightblue;}
.white {background-color: white;}
</style>
</head>

<body>

<h1 style="text-align: center;"><strong>줄넘, 귀!</strong></h1>
<p style="text-align: center;">요즘 많이 바쁘신가요?</p>
<p style="text-align: center;">줄넘귀는 바쁜 삶을 살고 있는 여러분을 위해 이동 시 빠르게 뉴스를 접할 수 있도록,</p>
<p style="text-align: center;">간략히 요약한 뉴스를 보이스를 통해 읽어주는 서비스를 제공합니다.</p>
<p>&nbsp;</p>

<form action="/topic" method = 'post'>
  <h3><strong><label for="press_nm">검색할 언론사:</label><br></h3>
  <p>현재 연합뉴스, MBC, 매일경제, SBS, KBS, JTBC 검색 기능을 지원합니다.</p>
  <input type="text" id="press_nm" name="press_nm" value=""><br>
  <h3><strong><label for="keyword">검색할 키워드:</label><br></h3>
  <input type="text" id="keyword" name="keyword" value=""><br>
  <h3><strong><label for="ct">수집 뉴스의 수(숫자만 입력) :</label><br></h3>
  <input type="int" id="ct" name="ct" value="3"><br><br>
  <input type="submit" value="Submit">
</form> 

<p>&nbsp;</p>
<p><strong>주요 서비스</strong></p>
<p>- 맞춤형 뉴스 선택 (언론사, 키워드 등)</p>
<p>- 긴 문서를 요약하여 핵심 문장으로 제공</p>
<p>- 고품질 음성 합성 기술로 65가지 음성과 한국어를 포함한 5개국 언어 제공</p>


</body>
</html>


Overwriting templates/inputnews2.html


In [7]:
%%writefile templates/resultnews2.html
<html>
<head>
<title>NEWS</title>
<style>
body {background-color: rgba(255, 255, 128, .5);}
.white {background-color: white;}
</style>
</head>
<body>
    <h1 style="text-align: center;"><strong>줄넘, 귀!</strong></h1>
    <p>&nbsp;</p>
    <h3><strong>검색할 언론사 : {{ press_nm }}</strong></h3>
    <p>&nbsp;</p>
    <h3><strong>검색할 키워드 : {{ keyword }}</strong></h3>
    <p>&nbsp;</p>
    <h3><strong>수집할 뉴스 개수 : {{ ct }}</strong></h3>
    <p>&nbsp;</p>
    <h3>뉴스</h3>
    <audio controls autoplay>
    <source src= {{ fn }} type="audio/mpeg">
        Your browser does not support the audio element.
    </audio>
    <h3><strong>요약 내용 : </strong></h3>
    <p>상세한 기사 내용은 하단 링크를 참조하세요.</p>
    <p>&nbsp;</p>
    <p>{{ news_list[0].replace('\n','') }}</p>
    <p>{{ news_list[1].replace('\n','') }}</p>    
    <p>{{ news_list[2].replace('\n','') }}</p>
    <p>{{ news_list[3].replace('\n','') }}</p>    
    <p>{{ news_list[4].replace('\n','') }}</p>    
    <p>{{ news_list[5].replace('\n','') }}</p>   

</body>
</html>



Overwriting templates/resultnews2.html


In [8]:
import sys, os
import requests
import selenium
from selenium import webdriver
import requests
from pandas import DataFrame
from bs4 import BeautifulSoup
import re
from datetime import datetime
import pickle, json, glob, time
from tqdm import tqdm

def process_a(press_nm, keyword, ct):
    ###### 날짜 저장 ##########
    date = str(datetime.now())
    date = date[:date.rfind(':')].replace(' ', '_')
    date = date.replace(':','시') + '분'
    sleep_sec = 0.5

    ####### 언론사별 본문 위치 태그 파싱 함수 ###########
    print('본문 크롤링에 필요한 함수를 로딩하고 있습니다...\n' + '-' * 100)
    def crawling_main_text(url):
        req = requests.get(url)
        req.encoding = None
        soup = BeautifulSoup(req.text, 'html.parser')
        
        # 연합뉴스
        if ('://yna' in url) | ('app.yonhapnews' in url):
            main_article = soup.find('article', {'class':'story-news article'})
            if main_article == None:
                main_article = soup.find('div', {'class' : 'article-txt'})
        
            text = main_article.text
        
        # MBC 
        elif '//imnews.imbc' in url: 
            text = soup.find('div', {'itemprop' : 'articleBody'}).text
        
        # 매일경제(미라클), req.encoding = None 설정 필요
        elif 'mirakle.mk' in url:
            text = soup.find('div', {'class' : 'view_txt'}).text
        
        # 매일경제, req.encoding = None 설정 필요
        elif 'mk.co' in url:
            text = soup.find('div', {'class' : 'art_txt'}).text
        
        # SBS
        elif 'news.sbs' in url:
            text = soup.find('div', {'itemprop' : 'articleBody'}).text
    
        # KBS
        elif 'news.kbs' in url:
            text = soup.find('div', {'id' : 'cont_newstext'}).text
        
        # JTBC
        elif 'news.jtbc' in url:
            text = soup.find('div', {'class' : 'article_content'}).text
        
        # 그 외
        else:
            text == None
        
        return text.replace('\n','').replace('\r','').replace('<br>','').replace('\t','')
    
    
    press_nm = '{}'.format(press_nm)

    print('검색할 언론사 : {}'.format(press_nm))


############### 브라우저를 켜고 검색 키워드 입력 ####################
   # query = input('검색할 키워드  : ')
   # news_num = int(input('수집 뉴스의 수(숫자만 입력) : '))

    print('\n' + '=' * 100 + '\n')

    print('브라우저를 실행시킵니다(자동 제어)\n')
    chrome_path = 'C:/Users/jhjh3/chromedriver_win32 (1)/chromedriver.exe'
    browser = webdriver.Chrome(chrome_path)

    news_url = 'https://search.naver.com/search.naver?where=news&query={}'.format(keyword)
    browser.get(news_url)
    time.sleep(sleep_sec)


######### 언론사 선택 및 confirm #####################
    print('설정한 언론사를 선택합니다.\n')

    search_opn_btn = browser.find_element_by_xpath('//a[@class="btn_option _search_option_open_btn"]')
    search_opn_btn.click()
    time.sleep(sleep_sec)

    bx_press = browser.find_element_by_xpath('//div[@role="listbox" and @class="api_group_option_sort _search_option_detail_wrap"]//li[@class="bx press"]')

# 기준 두번 째(언론사 분류순) 클릭하고 오픈하기
    press_tablist = bx_press.find_elements_by_xpath('.//div[@role="tablist" and @class="option"]/a')
    press_tablist[1].click()
    time.sleep(sleep_sec)

# 첫 번째 것(언론사 분류선택)
    bx_group = bx_press.find_elements_by_xpath('.//div[@class="api_select_option type_group _category_select_layer"]/div[@class="select_wrap _root"]')[0]

    press_kind_bx = bx_group.find_elements_by_xpath('.//div[@class="group_select _list_root"]')[0]
    press_kind_btn_list = press_kind_bx.find_elements_by_xpath('.//ul[@role="tablist" and @class="lst_item _ul"]/li/a')


    for press_kind_btn in press_kind_btn_list:
    
        # 언론사 종류를 순차적으로 클릭(좌측)
        press_kind_btn.click()
        time.sleep(sleep_sec)
    
        # 언론사선택(우측)
        press_slct_bx = bx_group.find_elements_by_xpath('.//div[@class="group_select _list_root"]')[1]
        # 언론사 선택할 수 있는 클릭 버튼
        press_slct_btn_list = press_slct_bx.find_elements_by_xpath('.//ul[@role="tablist" and @class="lst_item _ul"]/li/a')
        # 언론사 이름들 추출
        press_slct_btn_list_nm = [psl.text for psl in press_slct_btn_list]
    
        # 언론사 이름 : 언론사 클릭 버튼 인 딕셔너리 생성
        press_slct_btn_dict = dict(zip(press_slct_btn_list_nm, press_slct_btn_list))
    
        # 원하는 언론사가 해당 이름 안에 있는 경우
        # 1) 클릭하고
        # 2) 더이상 언론사분류선택 탐색 중지
        if press_nm in press_slct_btn_dict.keys():
            print('<{}> 카테고리에서 <{}>를 찾았으므로 탐색을 종료합니다'.format(press_kind_btn.text, press_nm))
        
            press_slct_btn_dict[press_nm].click()
            time.sleep(sleep_sec)
            
            break



################ 뉴스 크롤링 ########################
    print('\n크롤링을 시작합니다.')
    news_dict = {}
    idx = 1
    cur_page = 1

    pbar = tqdm(total=ct ,leave = True)
    
    while idx < ct:

        table = browser.find_element_by_xpath('//ul[@class="list_news"]')
        li_list = table.find_elements_by_xpath('./li[contains(@id, "sp_nws")]')
        area_list = [li.find_element_by_xpath('.//div[@class="news_area"]') for li in li_list]
        a_list = [area.find_element_by_xpath('.//a[@class="news_tit"]') for area in area_list]
 
        for n in a_list[:min(len(a_list), ct-idx+1)]:
            n_url = n.get_attribute('href')
            news_dict[idx] = {'title' : n.get_attribute('title'), 
                              'url' : n_url,
                              'text' : crawling_main_text(n_url)}
        
            idx += 1
            pbar.update(1)
        
        if idx < ct:
            cur_page +=1

            pages = browser.find_element_by_xpath('//div[@class="sc_page_inner"]')
            next_page_url = [p for p in pages.find_elements_by_xpath('.//a') if p.text == str(cur_page)][0].get_attribute('href')

            browser.get(next_page_url)
            time.sleep(sleep_sec)
        else:
            pbar.close()
        
            print('\n브라우저를 종료합니다.\n' + '=' * 100)
            time.sleep(0.7)
            browser.close()
            break

#### 데이터 전처리하기 ###################################################### 

    print('데이터프레임 변환\n')
    news_df = DataFrame(news_dict).T

    folder_path = os.getcwd()
    xlsx_file_name = '네이버뉴스_본문_{}개_{}_{}.xlsx'.format(ct, keyword, date)

    news_df.to_excel(xlsx_file_name)

    print('엑셀 저장 완료 | 경로 : {}\\{}\n'.format(folder_path, xlsx_file_name))

    os.startfile(folder_path)

    # print('=' * 100 + '\n결과물의 일부')
    news_df
    
    return news_df

In [9]:
import os
import sys
import requests
import json
import urllib.request
# import pandas as pd
# news_df = pd.read_excel('./news_df.xlsx')
news_list=[]
site = []

def process_b(news_df):
    len(news_df.title)
    client_id = "qj40wl5n0t"
    client_secret = "1xzXEYgwlsbv9RQxd1onUSByiyvtBDPFV9NEY1mv"

    for i in range(len(news_df.title)):
        headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
        "Content-Type": "application/json"
    }
        language = "ko" # Language of document (ko, ja )
        model = "news" # Model used for summaries (general, news)
        tone = "2" # Converts the tone of the summarized result. (0, 1, 2, 3)
        summaryCount = "3" # This is the number of sentences for the summarized document.
        url= "https://naveropenapi.apigw.ntruss.com/text-summary/v1/summarize" 
        title = ""
        content = news_df.text[i+1]
        data = {
            "document": {
            "title": title,
            "content" : content},
            "option": {
            "language": language,
            "model": model,
            "tone": tone,
            "summaryCount" : summaryCount
            }
        }
        # print(json.dumps(data, indent=4, sort_keys=True))
        response = requests.post(url, data=json.dumps(data), headers=headers)
        rescode = response.status_code
        if(rescode == 200):
            r1=response.text  
            print(r1)

        else:
            print("Error : " + response.text)

        r1
        site = news_df.url[i+1]

        r1d = json.loads(r1)
        r1r1 = r1d['summary']
    #    r1r1= r1.split('연합뉴스')[1]
        r1r1 = '{}번째 뉴스입니다. {} '.format(i+1, r1r1)
        news_list.append(r1r1)
        news_list.append(site)
        print(r1r1)
        print(news_list)            
    
        encText = urllib.parse.quote(r1r1)
        data = "speaker=njihun&volume=0&speed=0&pitch=0&format=mp3&text=" + encText;
        url = "https://naveropenapi.apigw.ntruss.com/tts-premium/v1/tts"
        request = urllib.request.Request(url)
        request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
        request.add_header("X-NCP-APIGW-API-KEY",client_secret)
        response = urllib.request.urlopen(request, data=data.encode('utf-8'))
        rescode = response.getcode()
        if(rescode==200):
            response_body = response.read()
            fn = 'voice{}.mp3'.format(i)
            with open(fn, 'wb') as f:
                f.write(response_body)
            print(fn)
        else:
            print("Error Code:" + rescode)
            
    result = (r1r1, fn)
    
    return news_list

In [10]:
### 요약부분 수정하는 함수 
def clean_news(news_list):
    remove = ['\n', '\\', '[사진 제공: 연합뉴스]', '앵커]', '[','사진=REUTERS 연합뉴스〉','사진 제공:연합뉴스','사진 제공: 연합뉴스]','사진=EPA 연합뉴스']
    for t in remove:
        for i, o in enumerate(news_list):
            news_list[i] = o.replace(t, " ")
   
    return news_list 

In [11]:
app=FastAPI()
templates = Jinja2Templates(directory="templates")
app.mount('/static', StaticFiles(directory='static'), name='static')

@app.get('/', response_class=HTMLResponse)
async def read_topic(request: Request):
    return templates.TemplateResponse("inputnews2.html", {"request": request})

@app.post('/topic', response_class=HTMLResponse)
async def get_topic(request: Request, press_nm: str = Form(...), keyword: str = Form(...), ct: int = Form(...)):
    news_df = process_a(press_nm, keyword, ct)
    news_list = process_b(news_df)
    news_list = clean_news(news_list) 
    audio1 = open("voice0.mp3", "rb").read()
    audio2 = open("voice1.mp3", "rb").read()
    audio3 = open("voice2.mp3", "rb").read()
    audioJoin = audio1 + audio2 + audio3 
    audioFinal = open("./static/audioFinal.mp3", "wb").write(audioJoin)
    # print(result)

    return templates.TemplateResponse("resultnews2.html", {"request": request,'press_nm': press_nm, 'keyword': keyword, 'ct':ct, 'news_list': news_list, 'fn': "/static/audioFinal.mp3"})


In [12]:
app.url_path_for('static', path='/audioFinal.mp3')

'/static/audioFinal.mp3'

In [13]:
!ngrok authtoken 26P5NSFkUtnURKy9pPsw14XpSaq_4PoTwUvNPG3XmHY3oY3VP

Authtoken saved to configuration file: C:\Users\jhjh3/.ngrok2/ngrok.yml


In [14]:
ngrok_tunnel = ngrok.connect(8000)
print ('Public URL:', ngrok_tunnel.public_url) 
nest_asyncio.apply()
uvicorn.run(app, host='0.0.0.0', port=8000)

INFO:     Started server process [28276]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: http://03c1-211-44-6-54.ngrok.io
INFO:     211.44.6.54:0 - "GET / HTTP/1.1" 200 OK
INFO:     182.172.56.199:0 - "GET / HTTP/1.1" 200 OK
INFO:     211.44.6.54:0 - "GET / HTTP/1.1" 200 OK
INFO:     211.44.6.54:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     211.44.6.54:0 - "GET / HTTP/1.1" 200 OK
본문 크롤링에 필요한 함수를 로딩하고 있습니다...
----------------------------------------------------------------------------------------------------
검색할 언론사 : JTBC


브라우저를 실행시킵니다(자동 제어)



C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_28276/4216336124.py:79: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(chrome_path)


설정한 언론사를 선택합니다.



C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_28276/4216336124.py:89: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_opn_btn = browser.find_element_by_xpath('//a[@class="btn_option _search_option_open_btn"]')
C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_28276/4216336124.py:93: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  bx_press = browser.find_element_by_xpath('//div[@role="listbox" and @class="api_group_option_sort _search_option_detail_wrap"]//li[@class="bx press"]')
C:\Users\jhjh3\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:426: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")


<방송/통신> 카테고리에서 <JTBC>를 찾았으므로 탐색을 종료합니다

크롤링을 시작합니다.


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_28276/4216336124.py:146: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  table = browser.find_element_by_xpath('//ul[@class="list_news"]')
C:\Users\jhjh3\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:393: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:07<00:00,  2.46s/it]



브라우저를 종료합니다.
데이터프레임 변환

엑셀 저장 완료 | 경로 : C:\Users\jhjh3\PJT_02\네이버뉴스_본문_3개_코로나_2022-03-29_13시14분.xlsx

{"summary":"사진=EPA 연합뉴스〉신종 코로나바이러스 감염증(코로나19)와 독감에 모두 감염된 사람의 중증 진행 가능성과 사망 위험이 코로나19에만 걸린 사람보다 2배 이상 높다는 연구 결과가 나왔습니다.\n28일(현지시간) 과학전문 매체 사이언스데일리 보도에 따르면 영국 에든버러대학교와 리버풀대학교, 임페리얼칼리지런던 등 공동 연구팀은 코로나19와 독감을 동시에 앓고 있는 성인 입원환자가 코로나19 또는 다른 바이러스에 감염된 환자에 비해 심각한 중증이나 사망 위험이 높았다고 밝혔습니다.\n연구팀은 지난 2020년 2월부터 2021년 12월까지 영국 병원에 입원한 성인 코로나19 환자 21만 2466명을 대상으로 인플루엔자 바이러스, 호흡기세포융합 바이러스(RSV) 또는 아데노바이러스 동시 감염 사례를 조사했습니다."}
1번째 뉴스입니다. 사진=EPA 연합뉴스〉신종 코로나바이러스 감염증(코로나19)와 독감에 모두 감염된 사람의 중증 진행 가능성과 사망 위험이 코로나19에만 걸린 사람보다 2배 이상 높다는 연구 결과가 나왔습니다.
28일(현지시간) 과학전문 매체 사이언스데일리 보도에 따르면 영국 에든버러대학교와 리버풀대학교, 임페리얼칼리지런던 등 공동 연구팀은 코로나19와 독감을 동시에 앓고 있는 성인 입원환자가 코로나19 또는 다른 바이러스에 감염된 환자에 비해 심각한 중증이나 사망 위험이 높았다고 밝혔습니다.
연구팀은 지난 2020년 2월부터 2021년 12월까지 영국 병원에 입원한 성인 코로나19 환자 21만 2466명을 대상으로 인플루엔자 바이러스, 호흡기세포융합 바이러스(RSV) 또는 아데노바이러스 동시 감염 사례를 조사했습니다. 
['1번째 뉴스입니다. 사진=EPA 연합뉴스〉신종 코로나바이러스 감염증(코로나19)와 독감에 모두 감염된 사람의 중증 진행 가능성과

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [28276]
